In [1]:
import json
from random import randrange
from sklearn.metrics import classification_report

In [2]:
with open('../../data/lyrics.json', 'r', encoding='utf8') as f:
    dataset = json.load(f)

In [4]:
## defining two dictionaries for bidirectional mapping between int & class
class_to_int = {'neutral' : 0, 'liebevoll' : 1, 'gewalttätig' : 2, 'rassistisch' : 3,
                   'homophob' : 4, 'frauenfeindlich' : 5, 'freundlich' : 6, 'positiv' : 7, 'traurig' : 8}
int_to_class = {v: k for k, v in class_to_int.items()}

In [5]:
def get_highest_class(song):
    max_class = 0
    highest_score = 0
    for key, value in song['total_class_score'].items():
        if highest_score < value:
            max_class = class_to_int[key]
            highest_score = value
    return max_class

In [30]:
## create a list where each element represents the highest scoring class for each song from 0 to n
n = 100
predictions = []
for song in dataset[100:200]:
    max_class = 0
    highest_score = 0
    for key, value in song['total_class_score'].items():
        if highest_score < value:
            max_class = class_to_int[key]
            highest_score = value
    predictions.append(max_class)
    
truth = [randrange(8) for i in range(0,100)]

In [9]:
kek = []
kek.append(dataset[0])
if not dataset[0] in kek:
    print("shit")

In [33]:
## randomly sample n songs for each class to a total of 9*n (9 classes)

import random


total = 0
n = 10
attempts = 0
samples = []
random.seed(237)

for i in range(9):
    elist = []
    samples.append(elist)
while total < 9*n and attempts < 1000:
    idx = randrange(len(dataset))
    class_number = get_highest_class(dataset[idx])
    if (not dataset[idx] in samples[class_number]) and (len(samples[class_number]) < n):
        dataset[idx]['top_class'] = class_number
        samples[class_number].append(dataset[idx])
        total += 1
    attempts += 1

In [38]:
count = 0
for list in samples:
    count += len(list)

count

count = 0
for list in samples:
    for song in list:
        if count == 35:
            print(song['genius_track_id'])
        count += 1


4948490


In [13]:
## we're interested in the macro-avg
print(classification_report(truth, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        17
           2       0.17      0.31      0.22        13
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00        14
           6       0.12      0.25      0.17         8
           7       0.10      0.33      0.15        12
           8       0.00      0.00      0.00         0

    accuracy                           0.10       100
   macro avg       0.04      0.10      0.06       100
weighted avg       0.04      0.10      0.06       100



c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Rec

In [28]:
## calculating the number of songs labeled per each class

class_scores = {0 : 0, 1 : 0, 2 : 0, 3 : 0,
                   4 : 0, 5 : 0, 6 : 0, 7 : 0, 8 : 0}

for song in dataset:
    max_class = 0
    highest_score = 0
    for key, value in song['total_class_score'].items():
        if highest_score < value:
            max_class = class_to_int[key]
            highest_score = value
    class_scores[max_class] += 1

## convert
class_scores = {int_to_class[k]: v for k, v in class_scores.items()}

In [29]:
class_scores

{'neutral': 167,
 'liebevoll': 227,
 'gewalttätig': 935,
 'rassistisch': 0,
 'homophob': 0,
 'frauenfeindlich': 76,
 'freundlich': 1732,
 'positiv': 2149,
 'traurig': 706}